# Kernel Shape Example
Spring 2019 AME-70790 Final Project

Nicholas Geneva (ngeneva@nd.edu)

Reference: Wand, M. P., & Jones, M. C. (1994). Kernel smoothing. Chapman and Hall/CRC.
___
In the previous example we looked at how the bandwidth of a kernel in significantly influence the kernel smoother prediction.
Now let us look at the impact of the shape of the kernel function $K$.
Typically, kernel functions are selected to be symmetrical.
Although there are some theoretical arguments for doing this, symmetric kernels are primarily used because they are easy to interpret.
To illustrate this let us again consider this arbitrary density:
\begin{equation}
    f_{1}(x)=\frac{3}{4}\phi\left(x | 0,1\right) + \frac{1}{4}\phi\left(x | 3/2, 1/3\right),
\end{equation}
where $\phi(x)$ is the normal PDF making this a mixture of two Gaussians. We will use 1000 training data points to approximate this density function with the kernel density estimator.

In [2]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

In [3]:
ntrain = 1000
band_widths = [0.06, 0.45, 0.15]

# Target data
x_test = np.linspace(-3,3,200)
y_test = 0.75*norm.pdf(x_test, loc=0, scale=1) + 0.75*norm.pdf(x_test, loc=3/2, scale=1/3)
# Training data
c1 = np.sum(np.random.rand(ntrain) < 0.75)
c2 = ntrain - c1
x_train0 = np.concatenate([np.random.randn(c1), (1/3.)*np.random.randn(c1)+1.5], axis=0)